# Using and creating Python packages

This tutorial will show how to use Python packages from Nixpkgs and create new ones. For reference material on Python in Nixpkgs, see the [Python section in the Nixpkgs manual](https://nixos.org/nixpkgs/manual#python)

## Python interpreters

Nixpkgs contains recipes for various Python interpreters. Multiple versions are available of the official reference implementation also known as CPython. Interpreters can be installed into a profile, used as part of a build or used temporary in a `nix-shell` or `nix run` session. 

## Python applications, libraries and environments

In Nixpkgs a distinction is made between applications, libraries and environments. 
- Applications are programs for end-users such as Calibre the e-book manager. 
- Libraries are packages providing importable modules. Additionally, tools used for development of libraries are also considered libraries.
- Environments are a Python interpreter along with libraries, typically used for development or as dependency for other programs.


## Python application

Let's consider first a Python application, the version control system Mercurial. The following recipe shows how to build it. For build and run-time dependencies we use the recipes provided by Nixpkgs. We then create a builder function for building our package. Finally, we call it.

In [ ]:
%%file default.nix

let
  nixpkgs = fetchTarball "channel:nixos-20.03";
  pkgs = import nixpkgs {};

  recipe = { python3, fetchurl }: 
    with python3.pkgs;
        
    buildPythonApplication rec {
      pname = "mercurial";
      version = "5.2.2";

      src = fetchurl {
        url = "https://mercurial-scm.org/release/mercurial-${version}.tar.gz";
        sha256 = "0fy00q0k4f0q64jjlnb7cl6m0sglivq9jgdddsp5sywc913zzigz";
      };
    
      # By default tests are executed, but they need to be invoked differently for this package
      dontUseSetuptoolsCheck = true;
    };

in pkgs.callPackage recipe {}


Let's build it. Note we suppress the build-output because of the amount.

In [ ]:
! nix-build --no-build-output

We can now execute `hg` directly from the store path

In [ ]:
! result/bin/hg --version

or install it in our profile

In [ ]:
! nix-env -if .

bringing the executable onto `$PATH`.

In [ ]:
! hg --version

Creating a basic recipe for Mercurial was easy. The recipe used in Nixpkgs is however more elaborate.

In [ ]:
! nix run -f channel:nixos-20.03 curl --command curl --silent https://raw.githubusercontent.com/NixOS/nixpkgs/nixos-20.03/pkgs/applications/version-management/mercurial/default.nix | pygmentize

## Python library

Python uses packages to distribute libraries called modules that are importable. In Nixpkgs all Python 3 libraries are in the attribute set `python3.pkgs`. The following example shows a recipe for the Python package `toolz`

In [ ]:
%%file toolz.nix

{ lib, buildPythonPackage, fetchPypi }:

buildPythonPackage rec {
  pname = "toolz";
  version = "0.10.0";

  src = fetchPypi {
    inherit pname version;
    sha256 = "08fdd5ef7c96480ad11c12d472de21acd32359996f69a5259299b540feba4560";
  };

  doCheck = false;

  meta = with lib; {
    homepage = https://github.com/pytoolz/toolz;
    description = "List processing tools and functional utilities";
    license = licenses.bsd3;
    maintainers = with maintainers; [ fridh ];
  };
}

Applications or other libraries may want to use this package. In order to make that possible, we need to update the Python packages set to include this package

In [ ]:
%%file mypython.nix

let
  nixpkgs = fetchTarball "channel:nixos-20.03";
  pkgs = import nixpkgs {};

  mypython = pkgs.python3.override {
    self = mypython;
    packageOverrides = pself: psuper: {
      toolz = pself.callPackage ./toolz.nix {};
    };
  };
in {
  inherit mypython;
}

We now have a Python interpreter that has a package set that includes our package. Using this type of overriding we can also inject modifications to existing packages that will then be used by all dependents in the set.

Let's build the package. 

In [ ]:
! nix-build -A mypython.pkgs.toolz mypython.nix

The log shows the execution of several hooks that are responsible for building, installing and testing the package.

## Environments

Now that we know how to build a Python library, we want to use it in an environment, for example a development environment.

The following expression creates a Python environment, quite similar to a `virtualenv` or `venv` that includes the interpreter, our package and another package, the test runner `pytest`

In [ ]:
%%file myenv.nix

(import ./mypython.nix).mypython.withPackages(ps: with ps; [ toolz pytest ])

In [ ]:
! nix run -f myenv.nix --command python3 -c 'import toolz; print(toolz); import pytest; print(pytest)'

The `toolz` module, just like the `pytest` module can now be imported.

How does an environment like this look like? If we build the expression we get the link to the store path

In [ ]:
! find $(nix-build myenv.nix) | tail

The environment is essentially a bunch of symbolic links to the packages we've included, as well as the interpreter. Executables are also wrapped so that all tools can find each other.